In [33]:
import geopandas as gpd
import numpy as np
import pandas as pd
import geojson_validator
from shapely.ops import unary_union
import pandas as pd
import numpy as np
from shapely.geometry import Point, MultiPolygon
import geopandas as gpd
from geopandas import GeoDataFrame
from fuzzywuzzy import process

In [34]:
rc_gdf = gpd.read_file(r"D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\Dashboards\risk-score\Assam\assam_rc_2024-11_reduced.json")
dist_gdf = gpd.read_file(r"D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\Dashboards\assam-flood-expenses\assam_district_2024-11_reduced.json")
mch = pd.read_csv(r'D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\Dashboards\MCH\MCH_tenderdata_Assam_2019_to_2025.csv')

In [35]:
# Function to fix invalid geometries
def fix_geometry(geom):
    if geom is None:
        return None
    if not geom.is_valid:
        # Attempt to fix using buffer(0)
        geom = geom.buffer(0)
    return geom if geom.is_valid else None

# Check and fix geometries
def clean_geometries(gdf):
    # Check for missing or invalid geometries
    gdf['geometry_fixed'] = gdf['geometry'].apply(fix_geometry)

    # Drop rows with irreparable (None) geometries
    gdf = gdf.dropna(subset=['geometry_fixed'])

    return gdf

def geometry_to_text(geom):
    if geom is None:
        return None
    
    # Handle Polygon and LineString geometries
    if geom.geom_type == 'Polygon':
        coords = list(geom.exterior.coords)
        return str([[lon, lat] for lon, lat in coords])
    
    elif geom.geom_type == 'LineString':
        coords = list(geom.coords)
        return str([[lon, lat] for lon, lat in coords])
    
    # Handle MultiPolygon and MultiLineString geometries
    elif geom.geom_type in ['MultiPolygon', 'MultiLineString']:
        all_coords = []
        for part in geom.geoms:  # Loop through each sub-geometry
            if part.geom_type == 'Polygon':
                coords = list(part.exterior.coords)
            else:
                coords = list(part.coords)
            all_coords.append([[lon, lat] for lon, lat in coords])
        return str(all_coords)
    
    # Catch other geometry types if necessary
    else:
        return None

def get_best_match(block_name, block_names):
    match, score = process.extractOne(block_name, block_names)
    return match if score > 80 else None  # Adjust threshold as needed

In [36]:
import json

# Flatten nested geometries if required
fixed_geo_flattened = {
    key: [json.dumps(item) if isinstance(item, dict) else item for item in value]
    for key, value in dist_gdf.items()
}
geo_fixed = pd.DataFrame.from_dict(fixed_geo_flattened)

In [5]:
# Simplify MultiPolygon by selecting the largest Polygon
def simplify_multipolygon(geometry):
    if isinstance(geometry, MultiPolygon):
        # Select the largest Polygon by area
        return max(geometry.geoms, key=lambda geom: geom.area)
    return geometry

# Apply the simplification
dist_gdf['geometry'] = dist_gdf['geometry'].apply(simplify_multipolygon)

geo_fixed = gpd.GeoDataFrame(dist_gdf, geometry='geometry')


In [37]:
# replace 'CONFLICT' in DISTRICT_FINALISED with tender_district_location before merge
mask = mch['DISTRICT_FINALISED'].notnull() & mch['DISTRICT_FINALISED'].astype(str).str.upper().eq('CONFLICT')
mch.loc[mask, 'DISTRICT_FINALISED'] = mch.loc[mask, 'tender_district_location']

# quick check
mch[['DISTRICT_FINALISED', 'tender_district_location']].head()

,DISTRICT_FINALISED,tender_district_location
0,KOKRAJHAR,KOKRAJHAR
1,KOKRAJHAR,KOKRAJHAR
2,KOKRAJHAR,KOKRAJHAR
3,KOKRAJHAR,KOKRAJHAR
4,KOKRAJHAR,KOKRAJHAR


In [38]:
merged_gdf = mch.merge(geo_fixed[['geometry','dtname']], right_on=['dtname'], left_on=['DISTRICT_FINALISED'], how='outer')
#merged_gdf = merged_gdf.drop(columns=['dtname','tender_revenueci_location'])
merged_gdf

,Tender ID,tender_externalreference,tender_title,Work Description,Tender Category,Tender Type,Form of contract,Product Category,Is Multi Currency Allowed For BOQ,Allow Two Stage Bidding,...,is_awarded,is_mch_broad,month,tender_district_externalReference,tender_district_title_description,tender_district_location,BTC_flag,DISTRICT_FINALISED,geometry,dtname
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POLYGON ((91.24956358328747 26.523283313596423...,BAJALI
1,2019_NHM_14251_1,NHM/C-CW/Labour Room_LaQshya/3839,Upgradation of Labour Room as per LaQshya guid...,Upgradation of Labour Room as per LaQshya guid...,Works,Open Tender,Item Rate,Civil Works,No,No,...,True,True,2021_03,NaN,BAKSA,BAKSA,False,BAKSA,POLYGON ((91.24956358328747 26.523283313596423...,BAKSA
2,2019_NHM_12797_2,NHM/CCW/Labor room Laqshya/3839/18/75,Upgradation of Labor room as per Laqshya Guide...,Upgradation of Labor room as per Laqshya Guide...,Works,Open Tender,Item Rate,Civil Works,No,No,...,False,True,2021_03,NaN,BARPETA,BARPETA,False,BARPETA,POLYGON ((91.22210213805252 26.179396358129598...,BARPETA
3,2020_NHM_16498_1,NHM/ C-CW/SNCU/FAAMCH/2897,UPGRADATION OF LABOUR ROOM IN BARPETA MEDICAL ...,UPGRADATION OF LABOUR ROOM IN BARPETA MEDICAL ...,Works,Open Tender,Turn-key,Civil Works - Building,No,No,...,False,True,2021_03,NaN,BARPETA,BARPETA,False,BARPETA,POLYGON ((91.22210213805252 26.179396358129598...,BARPETA
4,2023_HFWD_29759_1,FAAMCH/eTSNCUPICU/1850/23/398,SUPPLY OF DRUGS/SURGICALS and CONSUMABLES (rel...,SUPPLY OF DRUGS/SURGICALS and CONSUMABLES (rel...,Goods,Open Tender,Item Wise,Miscellaneous Goods,No,No,...,True,True,2023_05,NaN,BARPETA,BARPETA,False,BARPETA,POLYGON ((91.22210213805252 26.179396358129598...,BARPETA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,2019_NHM_13635_1,NRHM/CH/SNCU/Online Reporting/256,Printing and supply of SNCU Reporting Materials,Printing and supply of SNCU Reporting Materials,Goods,Open Limited,Supply,Publishing/Printing,No,No,...,False,True,2021_04,NaN,NaN,NaN,False,NaN,NaN,NaN
119,2019_NHM_13891_1,NHM/RCH Register/2961/2016-17/12645,Printing and Supply of RCH Register,Printing and Supply of RCH Register,Goods,Open Limited,Supply,Publishing/Printing,No,No,...,False,True,2021_04,NaN,NaN,NaN,False,NaN,NaN,NaN
120,2023_PWBNH_30838_1,JGWTBD_Repair_Renov._Rani CHC,Repairing and Renovation of Medicine Store Roo...,Repairing and Renovation of Medicine Store Roo...,Works,Open Tender,Item Wise,Civil Works - Building,No,No,...,False,True,2023_07,NaN,NaN,NaN,False,NaN,NaN,NaN
121,2024_HFWD_40161_1,JE Vaccine 30-10-2024,JE Vaccine,JE Vaccine Inactivated ( Absorbed Human),Goods,Single,Supply,Medicines,No,No,...,True,True,2025_02,NaN,NaN,NaN,False,NaN,NaN,NaN


In [31]:
merged_gdf['dtname'].nunique()

35

In [40]:
merged_gdf['polygons'] = merged_gdf['geometry'].apply(lambda x: geometry_to_text(x) if pd.notna(x) else None)
merged_gdf

,Tender ID,tender_externalreference,tender_title,Work Description,Tender Category,Tender Type,Form of contract,Product Category,Is Multi Currency Allowed For BOQ,Allow Two Stage Bidding,...,is_mch_broad,month,tender_district_externalReference,tender_district_title_description,tender_district_location,BTC_flag,DISTRICT_FINALISED,geometry,dtname,polygons
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POLYGON ((91.24956358328747 26.523283313596423...,BAJALI,"[[91.24956358328747, 26.523283313596423], [91...."
1,2019_NHM_14251_1,NHM/C-CW/Labour Room_LaQshya/3839,Upgradation of Labour Room as per LaQshya guid...,Upgradation of Labour Room as per LaQshya guid...,Works,Open Tender,Item Rate,Civil Works,No,No,...,True,2021_03,NaN,BAKSA,BAKSA,False,BAKSA,POLYGON ((91.24956358328747 26.523283313596423...,BAKSA,"[[91.24956358328747, 26.523283313596423], [91...."
2,2019_NHM_12797_2,NHM/CCW/Labor room Laqshya/3839/18/75,Upgradation of Labor room as per Laqshya Guide...,Upgradation of Labor room as per Laqshya Guide...,Works,Open Tender,Item Rate,Civil Works,No,No,...,True,2021_03,NaN,BARPETA,BARPETA,False,BARPETA,POLYGON ((91.22210213805252 26.179396358129598...,BARPETA,"[[91.22210213805252, 26.179396358129598], [91...."
3,2020_NHM_16498_1,NHM/ C-CW/SNCU/FAAMCH/2897,UPGRADATION OF LABOUR ROOM IN BARPETA MEDICAL ...,UPGRADATION OF LABOUR ROOM IN BARPETA MEDICAL ...,Works,Open Tender,Turn-key,Civil Works - Building,No,No,...,True,2021_03,NaN,BARPETA,BARPETA,False,BARPETA,POLYGON ((91.22210213805252 26.179396358129598...,BARPETA,"[[91.22210213805252, 26.179396358129598], [91...."
4,2023_HFWD_29759_1,FAAMCH/eTSNCUPICU/1850/23/398,SUPPLY OF DRUGS/SURGICALS and CONSUMABLES (rel...,SUPPLY OF DRUGS/SURGICALS and CONSUMABLES (rel...,Goods,Open Tender,Item Wise,Miscellaneous Goods,No,No,...,True,2023_05,NaN,BARPETA,BARPETA,False,BARPETA,POLYGON ((91.22210213805252 26.179396358129598...,BARPETA,"[[91.22210213805252, 26.179396358129598], [91...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,2019_NHM_13635_1,NRHM/CH/SNCU/Online Reporting/256,Printing and supply of SNCU Reporting Materials,Printing and supply of SNCU Reporting Materials,Goods,Open Limited,Supply,Publishing/Printing,No,No,...,True,2021_04,NaN,NaN,NaN,False,NaN,NaN,NaN,None
119,2019_NHM_13891_1,NHM/RCH Register/2961/2016-17/12645,Printing and Supply of RCH Register,Printing and Supply of RCH Register,Goods,Open Limited,Supply,Publishing/Printing,No,No,...,True,2021_04,NaN,NaN,NaN,False,NaN,NaN,NaN,None
120,2023_PWBNH_30838_1,JGWTBD_Repair_Renov._Rani CHC,Repairing and Renovation of Medicine Store Roo...,Repairing and Renovation of Medicine Store Roo...,Works,Open Tender,Item Wise,Civil Works - Building,No,No,...,True,2023_07,NaN,NaN,NaN,False,NaN,NaN,NaN,None
121,2024_HFWD_40161_1,JE Vaccine 30-10-2024,JE Vaccine,JE Vaccine Inactivated ( Absorbed Human),Goods,Single,Supply,Medicines,No,No,...,True,2025_02,NaN,NaN,NaN,False,NaN,NaN,NaN,None


In [41]:
from datetime import date, timedelta, datetime


#snapshot_ld_dist['datetime'] = pd.to_datetime(snapshot_ld_dist['timeperiod'], format='%Y_%m')
merged_gdf['timeperiod'] = merged_gdf['month'].str.replace('_', '-') #+ '-01'

# Step 2: Convert the modified column to datetime format
#snapshot_ld_dist['timeperiod_iso'] = pd.to_datetime(snapshot_ld_dist['timeperiod_iso'], format='%Y-%m-%d')
merged_gdf['timeperiod'] = pd.to_datetime(merged_gdf['timeperiod'], format='%Y-%m')


merged_gdf['timeperiod'] = merged_gdf['timeperiod'].dt.strftime('%Y-%m')

In [42]:
merged_gdf.columns

Index(['Tender ID', 'tender_externalreference', 'tender_title',
       'Work Description', 'Tender Category', 'Tender Type',
       'Form of contract', 'Product Category',
       'Is Multi Currency Allowed For BOQ', 'Allow Two Stage Bidding',
       'Independent External Monitor/Remarks', 'Published Date',
       'Pre Bid Meeting Date', 'Bid Validity(Days)', 'Should Allow NDA Tender',
       'Allow Preferential Bidder', 'Payment Mode', 'Bid Opening Date',
       'Organisation Chain', 'location', 'Pincode', 'No of Bids Received',
       'Tender Value in ₹', 'Bidder Name', 'Awarded Value', 'Status',
       'Contract Date :', 'Tender Stage', 'Department', 'is_mch_strict',
       'positive_keywords_dict', 'negative_keywords_dict', 'Scheme',
       'is_awarded', 'is_mch_broad', 'month',
       'tender_district_externalReference',
       'tender_district_title_description', 'tender_district_location',
       'BTC_flag', 'DISTRICT_FINALISED', 'geometry', 'dtname', 'polygons',
       'timeperi

In [43]:
merged_gdf['Awarded Value'] = merged_gdf['Awarded Value'].fillna(0)

In [44]:
merged_gdf.columns = merged_gdf.columns.str.lower().str.replace(' ', '_')
merged_gdf = merged_gdf.rename(columns={'contract_date_:':'contract_date','bid_validity(days)':'bid_validity_days','tender_value_in_₹':'tender_value_in_rupees'})
#merged_gdf['tender_value_in_rupees'] = merged_gdf['tender_value_in_rupees'].astype(float)
#merged_gdf = merged_gdf.dropna(subset=['awarded_value', 'district_finalised'])
merged_gdf['awarded_value'] = merged_gdf['awarded_value'].astype(float)
merged_gdf

,tender_id,tender_externalreference,tender_title,work_description,tender_category,tender_type,form_of_contract,product_category,is_multi_currency_allowed_for_boq,allow_two_stage_bidding,...,month,tender_district_externalreference,tender_district_title_description,tender_district_location,btc_flag,district_finalised,geometry,dtname,polygons,timeperiod
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,POLYGON ((91.24956358328747 26.523283313596423...,BAJALI,"[[91.24956358328747, 26.523283313596423], [91....",NaN
1,2019_NHM_14251_1,NHM/C-CW/Labour Room_LaQshya/3839,Upgradation of Labour Room as per LaQshya guid...,Upgradation of Labour Room as per LaQshya guid...,Works,Open Tender,Item Rate,Civil Works,No,No,...,2021_03,NaN,BAKSA,BAKSA,False,BAKSA,POLYGON ((91.24956358328747 26.523283313596423...,BAKSA,"[[91.24956358328747, 26.523283313596423], [91....",2021-03
2,2019_NHM_12797_2,NHM/CCW/Labor room Laqshya/3839/18/75,Upgradation of Labor room as per Laqshya Guide...,Upgradation of Labor room as per Laqshya Guide...,Works,Open Tender,Item Rate,Civil Works,No,No,...,2021_03,NaN,BARPETA,BARPETA,False,BARPETA,POLYGON ((91.22210213805252 26.179396358129598...,BARPETA,"[[91.22210213805252, 26.179396358129598], [91....",2021-03
3,2020_NHM_16498_1,NHM/ C-CW/SNCU/FAAMCH/2897,UPGRADATION OF LABOUR ROOM IN BARPETA MEDICAL ...,UPGRADATION OF LABOUR ROOM IN BARPETA MEDICAL ...,Works,Open Tender,Turn-key,Civil Works - Building,No,No,...,2021_03,NaN,BARPETA,BARPETA,False,BARPETA,POLYGON ((91.22210213805252 26.179396358129598...,BARPETA,"[[91.22210213805252, 26.179396358129598], [91....",2021-03
4,2023_HFWD_29759_1,FAAMCH/eTSNCUPICU/1850/23/398,SUPPLY OF DRUGS/SURGICALS and CONSUMABLES (rel...,SUPPLY OF DRUGS/SURGICALS and CONSUMABLES (rel...,Goods,Open Tender,Item Wise,Miscellaneous Goods,No,No,...,2023_05,NaN,BARPETA,BARPETA,False,BARPETA,POLYGON ((91.22210213805252 26.179396358129598...,BARPETA,"[[91.22210213805252, 26.179396358129598], [91....",2023-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,2019_NHM_13635_1,NRHM/CH/SNCU/Online Reporting/256,Printing and supply of SNCU Reporting Materials,Printing and supply of SNCU Reporting Materials,Goods,Open Limited,Supply,Publishing/Printing,No,No,...,2021_04,NaN,NaN,NaN,False,NaN,NaN,NaN,None,2021-04
119,2019_NHM_13891_1,NHM/RCH Register/2961/2016-17/12645,Printing and Supply of RCH Register,Printing and Supply of RCH Register,Goods,Open Limited,Supply,Publishing/Printing,No,No,...,2021_04,NaN,NaN,NaN,False,NaN,NaN,NaN,None,2021-04
120,2023_PWBNH_30838_1,JGWTBD_Repair_Renov._Rani CHC,Repairing and Renovation of Medicine Store Roo...,Repairing and Renovation of Medicine Store Roo...,Works,Open Tender,Item Wise,Civil Works - Building,No,No,...,2023_07,NaN,NaN,NaN,False,NaN,NaN,NaN,None,2023-07
121,2024_HFWD_40161_1,JE Vaccine 30-10-2024,JE Vaccine,JE Vaccine Inactivated ( Absorbed Human),Goods,Single,Supply,Medicines,No,No,...,2025_02,NaN,NaN,NaN,False,NaN,NaN,NaN,None,2025-02


In [46]:
merged_gdf['dtname'].nunique()

35

In [47]:
merged_gdf.to_csv(r'D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\Deployment\data-analysis-reference\Assam_MCH_analysis\TENDERS\data\Superset prep\MCH_tenderdata_Assam_2019_to_2025_polygons.csv', index=False)